In [0]:
# Importando módulo de seguimiento
import sys
import importlib
sys.path.append('/Workspace/Users/luisfdiaz@bcp.com.pe/Modulo de Seguimiento/')

import SegScore as rmm
importlib.reload(rmm)

# Librerias y fuciones generales
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim, broadcast, avg, max, min, lit, concat, window, round as colround, upper, abs as sparkabs,greatest
from pyspark.sql import functions as F
from pyspark import StorageLevel
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

In [0]:
def rezago_codmes_col(base, rezago_mes):
    for i in range(0, rezago_mes + 1, 1):
        base = base.withColumn(f"CODMES_{i}", date_format(add_months(to_date(concat(col("codmes"), lit("01")), "yyyyMMdd"), -i), "yyyyMM"))
        base = base.withColumn(f"CODMES_{i}", col(f"CODMES_{i}").cast("integer"))
    return base

def avance_codmes_col(base, avance_mes):
    for l in range(0, avance_mes + 1, 1):
        base = base.withColumn(f"CODMES_F{l}", date_format(add_months(to_date(concat(col("codmes"), lit("01")), "yyyyMMdd"), l), "yyyyMM"))
        base = base.withColumn(f"CODMES_F{l}", col(f"CODMES_F{l}").cast("integer"))
    return base

In [0]:
hipo = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_univ_pre_driver_app_hip')

In [0]:
hipo = hipo.withColumn('CODMES', col("CODMES").cast("integer"))

In [0]:
hipo = rezago_codmes_col(base = hipo, rezago_mes = 12)
hipo = avance_codmes_col(base = hipo, avance_mes = 12)

In [0]:
hipo.select('CODMES', 'CODMES_2').display()

In [0]:
universo = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_BASE_APP_HIP')
universo = universo.filter("TIPCREDITO NOT IN ('HVIM','HMIM','HSIC','HVIC')")

# Creamos marca de fondeo BCP
universo = universo.withColumn(
    "TIPO_FONDEO",
    when(col("TIPCREDITO").isin('TH01','TH02','HICC','RHIP','RREF','HICB','HIFS','HIBS','HIBK','HIPA','HIPD', 'HIPO', 'HIPP', 
		'HIPT','HIPR', 'HCPO', 'HITR'), "FONDEO BCP")
    .when(col("TIPCREDITO").isin('GAHI', 'GAHP', 'GAHR', 'GAPC'), "GAHI")
    .when(col("TIPCREDITO").isin('HITP', 'HMIV', 'HVIV', 'HSIV', 'HBIV'), "FONDEO MVIV")
    .otherwise("OTROS")
)

universo = universo.drop("FECINGRESOEMPLEADO", "FECINGRESOEMPLEADO_F", "FECRETIROEMPLEADO", "FECRETIROEMPLEADO_F")

In [0]:
!pip install oracledb
import oracledb
import pandas as pd

# Datos de conexión
username = "T39290"
password = "Bcp1234$"
dsn = "130.1.22.91:1521/BCPDW3"

# Configuramos la conexión a oracle
connection = oracledb.connect(user=username, password=password, dsn=dsn)

# Query de extracción
query = "SELECT * FROM ODS_V.MD_EMPLEADO"

chunks = pd.read_sql(query, connection, chunksize=3000)

# Unir todos los chunks en un solo DataFrame
md_empleado = pd.concat(chunks)

# Cerrar la conexión
connection.close()

# Resumiendo gremio
gremio = gremio.fillna(np.nan)
gremio = md_empleado.groupby('CODCLAVECIC').agg({ 'FECINGRESOEMPLEADO': 'min', 'FECRETIROEMPLEADO': 'max' }).reset_index()
gremio['FECINGRESOEMPLEADO_F'] = gremio['FECINGRESOEMPLEADO'].dt.year * 100 + gremio['FECINGRESOEMPLEADO'].dt.month
gremio['FECRETIROEMPLEADO_F'] = gremio['FECRETIROEMPLEADO'].dt.year * 100 + gremio['FECRETIROEMPLEADO'].dt.month

gremio = spark.createDataFrame(gremio)

In [0]:
universo1 = universo.join(gremio, on="CODCLAVECIC", how='left')
universo1 = universo1.withColumn(
    "FLG_GREMIO_F",
    when((col("CODMES")>=col("FECINGRESOEMPLEADO_F")) & ((col("CODMES")<=col("FECRETIROEMPLEADO_F")) | (col("FECRETIROEMPLEADO_F").isNull())), 1)
    .otherwise(0)
)


universo2 = universo1.withColumn(
    "CAMPANIA_AGRUPADA_NEW", 
    when(
        (col("CAMPANIA_AGRUPADA").isNotNull()) & (col("TIPCREDITO").isin('HTIV','HVIV', 'HVIC', 'HVIM', 'HBIV', 'HBIC','HSIV', 'HSIC', 'HMIV', 'HMIC', 'HMIM', 'HITP')), 
        "Ahorro Local"
    ).when(
        (col("CAMPANIA_AGRUPADA").isNotNull()) & (col("TIPCREDITO").isin('HCPO', 'HIPA', 'HIPD', 'HIPO', 'HIPP','HIPR', 'HIPT', 'HIBS', 'HIBK', 'HREF', 'RHIP', 'HICC', 'HICB', 'HITR', 'HIFS', 'TH01', 'TH02')), 
        "Tradicional"
    ).when(
        col("TIPCREDITO").isin('GAHI','GAHP','GAPC'), 
        "GAHI"
    ).otherwise(col("CAMPANIA_AGRUPADA"))
)

universo2 = universo2.filter("FLG_GREMIO_F = 0 AND CAMPANIA_AGRUPADA_NEW IN ('Tradicional', 'Pyme')")

## Importando variables cloud

### Evolución de la utilización promedio de líneas entre 6 y 24 meses

In [0]:
hm_matrizdeudorproducto = spark.table('catalog_lhcl_prod_bcp.bcp_ddv_matrizvariables_v.hm_matrizdeudorrccproducto')
hm_matrizdeudorproducto = hm_matrizdeudorproducto.select("CODCLAVEUNICOCLI", "CODMESANALISIS as CODMES", "rcc_pct_utl6_utl24_rt_u24")
hm_matrizdeudorproducto = hm_matrizdeudorproducto.dropDuplicates(subset=["CODCLAVEUNICOCLI"])

### Evolución del promedio de deuda no hipotecaria entre 6 y 24 meses

In [0]:
hm_matrizexperienciacliente = spark.table('catalog_lhcl_prod_bcp.bcp_ddv_matrizvariables_v.hm_matrizexperienciacliente')
hm_matrizexperienciacliente = hm_matrizdeudorproducto.select("CODCLAVEUNICOCLI", "CODMES", "exp_pct_evol_ship_u6m_rt_u24")
hm_matrizexperienciacliente = hm_matrizexperienciacliente.dropDuplicates(subset=["CODCLAVEUNICOCLI"])

### Ratio entre el pasivo mínimo y el pasivo promedio en los últ. 24 meses

In [0]:
hm_matrizresumensaldoactpas = spark.table('catalog_lhcl_prod_bcp.bcp_ddv_matrizvariables_v.hm_matrizresumensaldoactivopasivo')
hm_matrizresumensaldoactpas = hm_matrizdeudorproducto.selectExpr("CODCLAVEUNICOCLI", "CODMESANALISIS as CODMES", "PROD_MTO_SLD_PRM_PAS_MIN_24_24_RT_U24 as SLD_PRM_PAS_MIN_24_24_RT_U24")
hm_matrizexperienciacliente = hm_matrizexperienciacliente.dropDuplicates(subset=["CODCLAVEUNICOCLI"])

In [0]:
universo2 = universo2.dropDuplicates(subset=['CODSOLICITUD'])
universo3 = universo2.join(
  hm_matrizdeudorproducto,
  on=['CODCLAVEUNICOCLI'],
  how='left'
)

universo4 = universo3.join(
  hm_matrizexperienciacliente,
  on=['CODCLAVEUNICOCLI'],
  how='left'
)

universo5 = universo4.join(
  hm_matrizresumensaldoactpas,
  on=['CODCLAVEUNICOCLI'],
  how='left'
)